# Classification on Clip

### Setup

In [1]:
%load_ext autoreload
%autoreload 2

### Get Data

In [2]:
import pickle
import numpy as np

In [3]:
X_splits_path = 'cifar100/meru_projection_normal/image_feats.pickle'
y_splits_path = 'cifar100/meru_projection_normal/labels.pickle'
curvature = -0.10000000149011612

In [4]:
# load data and seperate into train, val, test

subset = 20

with open(X_splits_path, 'rb') as f:
    X_splits = pickle.load(f)
    
with open(y_splits_path, 'rb') as f:
    y_splits = pickle.load(f)
    
X_train, X_val, X_test = X_splits['train'].numpy(), X_splits['val'].numpy(), X_splits['test'].numpy()
y_train, y_val, y_test = y_splits['train'].numpy(), y_splits['val'].numpy(), y_splits['test'].numpy()

if subset:
    X_train, X_val, X_test = X_train[:subset], X_val[:subset], X_test[:subset]
    y_train, y_val, y_test = y_train[:subset], y_val[:subset], y_test[:subset]

In [5]:
# get time components on hyperboloid (we only loaded space components)

def get_time(X, curvature=1):
    curvature = abs(curvature)
    x_time = np.sqrt(1 / curvature + np.sum(X**2, axis=-1))
    return x_time

X_train_time = get_time(X_train, curvature)
X_val_time = get_time(X_val, curvature)
X_test_time = get_time(X_test, curvature)

X_train = np.concatenate([X_train_time[:, None], X_train], axis=-1)
X_val = np.concatenate([X_val_time[:, None], X_val], axis=-1)
X_test = np.concatenate([X_test_time[:, None], X_test], axis=-1)

### Train Model

In [6]:
from sklearn.metrics import accuracy_score
from src.hyperdt.tree import HyperbolicDecisionTreeClassifier, DecisionTreeClassifier

In [7]:
dt = DecisionTreeClassifier(max_depth=3)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_val)
dt_acc = accuracy_score(y_val, y_pred)
dt_acc

0.0

In [8]:
hdt = HyperbolicDecisionTreeClassifier(max_depth=3, curvature=curvature)
hdt.fit(X_train, y_train)
y_pred = hdt.predict(X_val)
hdt_acc = accuracy_score(y_val, y_pred)
hdt_acc

0.05